In [1]:
# !pip install livelossplot -q
# !pip install bposd -q
# !pip install json -q

In [2]:
import os
import sys
sys.path.append(os.getcwd() + "/src")

## Benchmark random CPC codes

In [3]:
from scoring.score_dataset import run_decoder
import json
from global_params import params
from CPC import generate_random
import numpy as np
import utils

error_rates = params['eval_p_range']
n_samples = 100
best_word_succ_rates = [0.0] * len(error_rates)

for i, e in enumerate(error_rates):
	for _ in range(n_samples):
		pc, _, _, _ = generate_random.random_cpc()
		wsr = run_decoder(pc, 100_000, np.ones(pc.shape[-1]) * 0.001)
		if wsr > best_word_succ_rates[i]:
			best_word_succ_rates[i] = wsr

out_json = {'random_cpc': {}}
for i, r in enumerate(best_word_succ_rates):
	out_json['random_cpc'][f"error_rate_{error_rates[i]}"] = r


with open(utils.get_eval_baseline_path(), "w") as outfile:
    print("WRITING", out_json)
    json.dump(out_json, outfile, cls=utils.NpEncoder)

/home/lev/code/research/ai/AI_QEC/src/scoring/score_dataset.py:53: RuntimeWarning: invalid value encountered in double_scalars
  if bp_frame_error_rate_eb / (1-p) < err_bar_cutoff:


WRITING {'random_cpc': {'error_rate_0.001': 0.99807, 'error_rate_0.005': 0.99795, 'error_rate_0.01': 0.99795, 'error_rate_0.015': 0.99794}}


## Benchmark AI learned codes